# NCAAM Handicap Spread Coverage Network Rating

## Objective
Build network-based strength ratings using **handicap-adjusted spread coverage**.
This applies an 11-point cushion from each team's perspective to identify:
- **Cushion-Dependent Teams**: Need the handicap to cover
- **Cushion-Proof Teams**: Cover comfortably regardless of handicap

## Key Concept
- Home covers with handicap if: `spread_result_diff >= -11`
- Away covers with handicap if: `spread_result_diff <= 11`
- Games within ±11 points → both teams "cover with handicap"
- Network edge goes to team with larger handicap-adjusted margin

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# NCAAM Handicap Spread Config
CONFIG = {
    'sport': 'NCAAM',
    'handicap': 11,          # Point cushion applied to each team
    'margin_cap': 12,        # College variance slightly higher
    'recency_decay': 0.92,
    'learning_rate': 0.08,
    'iterations': 100,
    'tolerance': 0.001
}

print(f"Handicap Spread Config: {CONFIG}")

## Phase 1: Data Loading

In [ ]:
# Load NCAAM data
data_file = Path().resolve().parent / 'data' / 'results' / 'ncaam_season_results.xlsx'
df = pd.read_excel(data_file)

print(f"Loaded {len(df)} NCAAM games")
print(f"Date range: {df['game_date'].min().date()} to {df['game_date'].max().date()}")

# Filter to games with spread data
df_spread = df[df['spread_result_difference'].notna()].copy()
print(f"\nGames with spread data: {len(df_spread)}")

# Get all teams
all_teams = set(df_spread['home_team'].unique()) | set(df_spread['away_team'].unique())
print(f"Total teams: {len(all_teams)}")

df_spread.head()

## Phase 2: Handicap-Adjusted Cover Determination

In [ ]:
handicap = CONFIG['handicap']

# Calculate handicap-adjusted margins from each team's perspective
df_spread['home_margin_handicap'] = df_spread['spread_result_difference'] + handicap
df_spread['away_margin_handicap'] = handicap - df_spread['spread_result_difference']

# Determine if each team covers with the handicap
df_spread['home_covers_handicap'] = df_spread['spread_result_difference'] >= -handicap
df_spread['away_covers_handicap'] = df_spread['spread_result_difference'] <= handicap

# For network: cover_team is the one with larger handicap-adjusted margin
df_spread['cover_team'] = np.where(
    df_spread['home_margin_handicap'] >= df_spread['away_margin_handicap'],
    df_spread['home_team'],
    df_spread['away_team']
)
df_spread['fail_team'] = np.where(
    df_spread['home_margin_handicap'] >= df_spread['away_margin_handicap'],
    df_spread['away_team'],
    df_spread['home_team']
)
df_spread['cover_margin'] = np.maximum(
    df_spread['home_margin_handicap'],
    df_spread['away_margin_handicap']
)

# Summary statistics
print(f"Handicap Analysis ({handicap}pt cushion):")
print(f"  Games where BOTH teams cover with handicap: {(df_spread['home_covers_handicap'] & df_spread['away_covers_handicap']).sum()}")
print(f"  Games where only home covers: {(df_spread['home_covers_handicap'] & ~df_spread['away_covers_handicap']).sum()}")
print(f"  Games where only away covers: {(~df_spread['home_covers_handicap'] & df_spread['away_covers_handicap']).sum()}")
print(f"  Games where neither covers: {(~df_spread['home_covers_handicap'] & ~df_spread['away_covers_handicap']).sum()}")

df_spread[['game_date', 'home_team', 'away_team', 'spread_result_difference', 
           'home_margin_handicap', 'away_margin_handicap', 'cover_team', 'cover_margin']].head(10)

In [ ]:
# Visualize handicap coverage distribution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Original spread result distribution with handicap thresholds
ax = axes[0]
ax.hist(df_spread['spread_result_difference'], bins=40, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='blue', linestyle='-', linewidth=2, label='0pt (original threshold)')
ax.axvline(x=-handicap, color='green', linestyle='--', linewidth=2, label=f'-{handicap}pt (home handicap)')
ax.axvline(x=handicap, color='red', linestyle='--', linewidth=2, label=f'+{handicap}pt (away handicap)')
ax.axvspan(-handicap, handicap, alpha=0.2, color='yellow', label='Both cover zone')
ax.set_xlabel('Spread Result Difference')
ax.set_ylabel('Frequency')
ax.set_title(f'Spread Results with {handicap}pt Handicap Zones')
ax.legend(fontsize=8)

# Cover margin distribution (handicap-adjusted)
ax = axes[1]
ax.hist(df_spread['cover_margin'], bins=30, edgecolor='black', alpha=0.7, color='green')
ax.axvline(x=CONFIG['margin_cap'], color='r', linestyle='--', label=f"Margin Cap ({CONFIG['margin_cap']}pts)")
ax.set_xlabel('Handicap-Adjusted Cover Margin')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Cover Margins (with handicap)')
ax.legend()

# Compare: original vs handicap cover rates
ax = axes[2]
original_home_cover = (df_spread['spread_result_difference'] >= 0).mean() * 100
original_away_cover = (df_spread['spread_result_difference'] < 0).mean() * 100
handicap_home_cover = df_spread['home_covers_handicap'].mean() * 100
handicap_away_cover = df_spread['away_covers_handicap'].mean() * 100

x = np.arange(2)
width = 0.35
ax.bar(x - width/2, [original_home_cover, original_away_cover], width, label='0pt (Original)', color='#3498db')
ax.bar(x + width/2, [handicap_home_cover, handicap_away_cover], width, label=f'{handicap}pt Handicap', color='#2ecc71')
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax.set_xticks(x)
ax.set_xticklabels(['Home', 'Away'])
ax.set_ylabel('Cover Rate (%)')
ax.set_title('Cover Rates: Original vs Handicap')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nOriginal Cover Rates: Home {original_home_cover:.1f}%, Away {original_away_cover:.1f}%")
print(f"Handicap Cover Rates: Home {handicap_home_cover:.1f}%, Away {handicap_away_cover:.1f}%")

## Phase 3: Build Handicap Network & Compute Ratings

In [ ]:
def build_handicap_network(games_df, recency_decay=0.92):
    """
    Build weighted directed graph from handicap-adjusted spread coverage.
    Edge: cover_team -> fail_team (based on larger handicap margin)
    """
    G = nx.DiGraph()
    
    for team in all_teams:
        G.add_node(team)
    
    games_sorted = games_df.sort_values('game_date')
    max_date = games_sorted['game_date'].max()
    
    edge_data = {}
    
    for _, game in games_sorted.iterrows():
        cover_team = game['cover_team']
        fail_team = game['fail_team']
        cover_margin = game['cover_margin']
        
        days_ago = (max_date - game['game_date']).days
        recency_weight = recency_decay ** (days_ago / 7)
        
        key = (cover_team, fail_team)
        if key not in edge_data:
            edge_data[key] = {
                'games': 0,
                'total_margin': 0,
                'weighted_margin': 0,
                'total_weight': 0
            }
        
        edge_data[key]['games'] += 1
        edge_data[key]['total_margin'] += cover_margin
        edge_data[key]['weighted_margin'] += cover_margin * recency_weight
        edge_data[key]['total_weight'] += recency_weight
    
    for (cover_team, fail_team), data in edge_data.items():
        G.add_edge(cover_team, fail_team,
                   games=data['games'],
                   avg_margin=data['total_margin'] / data['games'],
                   weighted_margin=data['weighted_margin'] / data['total_weight'])
    
    return G

G_handicap = build_handicap_network(df_spread, CONFIG['recency_decay'])

print(f"Handicap Network Summary:")
print(f"  Nodes: {G_handicap.number_of_nodes()}")
print(f"  Edges: {G_handicap.number_of_edges()}")
print(f"  Density: {nx.density(G_handicap):.3f}")

In [ ]:
def compute_handicap_rating(games_df, max_iterations=100, tolerance=0.001, 
                            margin_cap=12, learning_rate=0.08):
    """
    Compute network-propagated handicap spread coverage ratings.
    """
    teams = set(games_df['home_team']) | set(games_df['away_team'])
    ratings = {team: 0.5 for team in teams}
    history = []
    
    for iteration in range(max_iterations):
        new_ratings = ratings.copy()
        
        for _, game in games_df.iterrows():
            cover_team = game['cover_team']
            fail_team = game['fail_team']
            cover_margin = min(game['cover_margin'], margin_cap)
            
            total = ratings[cover_team] + ratings[fail_team]
            expected = ratings[cover_team] / total if total > 0 else 0.5
            surprise = 1 - expected
            adjustment = surprise * (cover_margin / margin_cap) * learning_rate
            
            new_ratings[cover_team] += adjustment
            new_ratings[fail_team] -= adjustment
        
        min_r, max_r = min(new_ratings.values()), max(new_ratings.values())
        if max_r > min_r:
            new_ratings = {t: (r - min_r) / (max_r - min_r) for t, r in new_ratings.items()}
        
        max_change = max(abs(new_ratings[t] - ratings[t]) for t in teams)
        history.append(max_change)
        
        if max_change < tolerance:
            print(f"Converged at iteration {iteration + 1}")
            break
        
        ratings = new_ratings
    
    return ratings, history

handicap_ratings, convergence = compute_handicap_rating(
    df_spread,
    max_iterations=CONFIG['iterations'],
    tolerance=CONFIG['tolerance'],
    margin_cap=CONFIG['margin_cap'],
    learning_rate=CONFIG['learning_rate']
)

# Display top teams
handicap_df = pd.DataFrame([
    {'team': team, 'handicap_rating': rating}
    for team, rating in handicap_ratings.items()
]).sort_values('handicap_rating', ascending=False)

print("\nTop 10 Teams by Handicap Rating:")
print(handicap_df.head(10).to_string(index=False))

## Phase 4: Compare to Non-Handicap (0pt) Rating

In [ ]:
# Compute original (0pt) spread coverage rating for comparison
df_original = df_spread.copy()
df_original['cover_team'] = np.where(
    df_original['spread_result_difference'] >= 0,
    df_original['home_team'],
    df_original['away_team']
)
df_original['fail_team'] = np.where(
    df_original['spread_result_difference'] >= 0,
    df_original['away_team'],
    df_original['home_team']
)
df_original['cover_margin'] = df_original['spread_result_difference'].abs()

original_ratings, _ = compute_handicap_rating(
    df_original,
    max_iterations=CONFIG['iterations'],
    tolerance=CONFIG['tolerance'],
    margin_cap=CONFIG['margin_cap'],
    learning_rate=CONFIG['learning_rate']
)

print("Original (0pt) spread ratings computed.")

In [ ]:
# Build comparison dataframe
team_stats = []

for team in all_teams:
    home_games = df_spread[df_spread['home_team'] == team]
    away_games = df_spread[df_spread['away_team'] == team]
    total_games = len(home_games) + len(away_games)
    
    # Original cover stats
    orig_home_covers = (home_games['spread_result_difference'] >= 0).sum()
    orig_away_covers = (away_games['spread_result_difference'] < 0).sum()
    original_covers = orig_home_covers + orig_away_covers
    
    # Handicap cover stats
    hcap_home_covers = home_games['home_covers_handicap'].sum()
    hcap_away_covers = away_games['away_covers_handicap'].sum()
    handicap_covers = hcap_home_covers + hcap_away_covers
    
    team_stats.append({
        'team': team,
        'games': total_games,
        'original_covers': original_covers,
        'original_cover_pct': original_covers / total_games if total_games > 0 else 0,
        'handicap_covers': handicap_covers,
        'handicap_cover_pct': handicap_covers / total_games if total_games > 0 else 0,
        'original_rating': original_ratings.get(team, 0.5),
        'handicap_rating': handicap_ratings.get(team, 0.5)
    })

df_compare = pd.DataFrame(team_stats)

# Calculate key metrics
df_compare['handicap_delta'] = df_compare['handicap_rating'] - df_compare['original_rating']
df_compare['cover_pct_gain'] = df_compare['handicap_cover_pct'] - df_compare['original_cover_pct']

df_compare = df_compare.sort_values('handicap_delta', ascending=False)

print(f"Rating Comparison: Original (0pt) vs Handicap ({handicap}pt)")
print("="*90)
print(df_compare[['team', 'original_cover_pct', 'handicap_cover_pct', 'cover_pct_gain',
                  'original_rating', 'handicap_rating', 'handicap_delta']].head(20).to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Scatter: Original vs Handicap Rating
ax = axes[0]
scatter = ax.scatter(df_compare['original_rating'], df_compare['handicap_rating'],
                     c=df_compare['handicap_delta'], cmap='RdYlGn', s=50, alpha=0.6)
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Equal ratings')
ax.set_xlabel('Original Rating (0pt)')
ax.set_ylabel(f'Handicap Rating ({handicap}pt)')
ax.set_title('Original vs Handicap Rating\n(Color = Handicap Delta)')
fig.colorbar(scatter, ax=ax, label='Handicap Delta')

# Histogram of Handicap Deltas
ax = axes[1]
ax.hist(df_compare['handicap_delta'], bins=30, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.axvline(x=df_compare['handicap_delta'].mean(), color='red', linestyle='--', 
           label=f"Mean: {df_compare['handicap_delta'].mean():.3f}")
ax.set_xlabel('Handicap Delta')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Handicap Deltas')
ax.legend()

# Top/Bottom teams by delta
ax = axes[2]
top_bottom = pd.concat([df_compare.head(10), df_compare.tail(10)])
colors = ['#2ecc71' if d > 0 else '#e74c3c' for d in top_bottom['handicap_delta']]
ax.barh(top_bottom['team'], top_bottom['handicap_delta'], color=colors)
ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('Handicap Delta')
ax.set_title(f'Top/Bottom 10 Teams by Handicap Delta')

plt.tight_layout()
plt.show()

## Phase 5: Classify Teams (Cushion-Dependent vs Cushion-Proof)

In [ ]:
# Classify teams based on handicap delta (use percentile for NCAAM due to many teams)
top_pct = df_compare['handicap_delta'].quantile(0.9)
bottom_pct = df_compare['handicap_delta'].quantile(0.1)

cushion_dependent = df_compare[df_compare['handicap_delta'] >= top_pct].sort_values('handicap_delta', ascending=False)
cushion_proof = df_compare[df_compare['handicap_delta'] <= bottom_pct].sort_values('handicap_delta')
neutral = df_compare[(df_compare['handicap_delta'] > bottom_pct) & 
                     (df_compare['handicap_delta'] < top_pct)]

print("="*80)
print(f"TEAM CLASSIFICATION ({handicap}pt Handicap Analysis)")
print("="*80)

print(f"\n\U0001F7E1 CUSHION-DEPENDENT TEAMS (Top 10% by delta, {len(cushion_dependent)} teams)")
print(f"These teams improve significantly with the {handicap}pt cushion.")
print("They may be underperforming against the spread but competitive in games.")
print("-"*80)
if len(cushion_dependent) > 0:
    print(cushion_dependent[['team', 'games', 'original_cover_pct', 'handicap_cover_pct', 
                             'original_rating', 'handicap_rating', 'handicap_delta']].head(15).to_string(index=False))

print(f"\n\U0001F7E2 CUSHION-PROOF TEAMS (Bottom 10% by delta, {len(cushion_proof)} teams)")
print(f"These teams perform WORSE relative to others with the {handicap}pt cushion.")
print("They already cover convincingly - the cushion helps their opponents more.")
print("-"*80)
if len(cushion_proof) > 0:
    print(cushion_proof[['team', 'games', 'original_cover_pct', 'handicap_cover_pct', 
                         'original_rating', 'handicap_rating', 'handicap_delta']].head(15).to_string(index=False))

print(f"\n\U00002B1C NEUTRAL TEAMS ({len(neutral)} teams)")
print(f"These teams' relative rankings don't change much with the {handicap}pt cushion.")

## Phase 6: Predictive Analysis

In [ ]:
# Split data for predictive testing
df_sorted = df_spread.sort_values('game_date')
split_idx = int(len(df_sorted) * 0.75)

train_df = df_sorted.iloc[:split_idx].copy()
test_df = df_sorted.iloc[split_idx:].copy()

print(f"Training: {len(train_df)} games")
print(f"Testing: {len(test_df)} games")

# Compute handicap ratings on training data
train_df['cover_team'] = np.where(
    train_df['home_margin_handicap'] >= train_df['away_margin_handicap'],
    train_df['home_team'],
    train_df['away_team']
)
train_df['fail_team'] = np.where(
    train_df['home_margin_handicap'] >= train_df['away_margin_handicap'],
    train_df['away_team'],
    train_df['home_team']
)
train_df['cover_margin'] = np.maximum(train_df['home_margin_handicap'], train_df['away_margin_handicap'])

train_ratings, _ = compute_handicap_rating(train_df, margin_cap=CONFIG['margin_cap'])

# Calculate test period cover rates (with handicap)
# Only include teams with at least 3 games in test period
test_cover_rates = {}
for team in all_teams:
    home_test = test_df[test_df['home_team'] == team]
    away_test = test_df[test_df['away_team'] == team]
    
    total = len(home_test) + len(away_test)
    if total >= 3:
        covers = home_test['home_covers_handicap'].sum() + away_test['away_covers_handicap'].sum()
        test_cover_rates[team] = covers / total

# Correlation
pred_df = pd.DataFrame([
    {'team': t, 'train_rating': train_ratings.get(t, 0.5), 'test_cover': test_cover_rates.get(t, np.nan)}
    for t in all_teams if t in test_cover_rates and t in train_ratings
])

corr = pred_df['train_rating'].corr(pred_df['test_cover'])
print(f"\nTeams with 3+ test games: {len(pred_df)}")
print(f"Predictive Correlation (Training Handicap Rating vs Test Cover Rate): {corr:.3f}")

In [ ]:
# Visualize predictive power
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.scatter(pred_df['train_rating'], pred_df['test_cover'], alpha=0.5, s=40)
ax.set_xlabel('Training Handicap Rating')
ax.set_ylabel('Test Period Handicap Cover Rate')
ax.set_title(f'Predictive Power ({handicap}pt Handicap)\nCorrelation: {corr:.3f}')

z = np.polyfit(pred_df['train_rating'], pred_df['test_cover'], 1)
p = np.poly1d(z)
ax.plot([0, 1], [p(0), p(1)], 'r--', label=f'Trend')
ax.legend()

# Tier analysis
ax = axes[1]
pred_df['tier'] = pd.qcut(pred_df['train_rating'], q=4, labels=['Bottom 25%', 'Lower Mid', 'Upper Mid', 'Top 25%'])
tier_cover = pred_df.groupby('tier')['test_cover'].mean()

colors = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']
bars = ax.bar(tier_cover.index, tier_cover.values * 100, color=colors)
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Test Period Handicap Cover Rate (%)')
ax.set_title(f'Future Cover Rate by Training Tier ({handicap}pt Handicap)')
ax.tick_params(axis='x', rotation=15)

for bar, val in zip(bars, tier_cover.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val*100:.1f}%', ha='center')

plt.tight_layout()
plt.show()

## Summary

In [ ]:
print("="*70)
print(f"NCAAM {handicap}PT HANDICAP SPREAD NETWORK ANALYSIS: SUMMARY")
print("="*70)

print(f"\n1. DATA")
print(f"   - Games analyzed: {len(df_spread)}")
print(f"   - Teams: {len(all_teams)}")
print(f"   - Handicap: {handicap} points")

print(f"\n2. COVER RATE CHANGES")
print(f"   - Original home cover rate: {original_home_cover:.1f}%")
print(f"   - Handicap home cover rate: {handicap_home_cover:.1f}%")
print(f"   - Games where both cover with handicap: {(df_spread['home_covers_handicap'] & df_spread['away_covers_handicap']).sum()}")

rating_corr = df_compare['original_rating'].corr(df_compare['handicap_rating'])
print(f"\n3. RATING CORRELATION")
print(f"   - Original vs Handicap rating correlation: {rating_corr:.3f}")
print(f"   - Mean handicap delta: {df_compare['handicap_delta'].mean():.4f}")
print(f"   - Std handicap delta: {df_compare['handicap_delta'].std():.4f}")

print(f"\n4. TEAM CLASSIFICATION")
print(f"   - Cushion-Dependent (top 10%): {len(cushion_dependent)} teams")
print(f"   - Cushion-Proof (bottom 10%): {len(cushion_proof)} teams")
print(f"   - Neutral: {len(neutral)} teams")

if len(cushion_dependent) > 0:
    top_dependent = cushion_dependent.iloc[0]
    print(f"   - Most cushion-dependent: {top_dependent['team']} (delta: +{top_dependent['handicap_delta']:.3f})")
if len(cushion_proof) > 0:
    top_proof = cushion_proof.iloc[0]
    print(f"   - Most cushion-proof: {top_proof['team']} (delta: {top_proof['handicap_delta']:.3f})")

print(f"\n5. PREDICTIVE POWER")
print(f"   - Training-to-Test correlation: {corr:.3f}")

In [ ]:
# Export results
export_df = df_compare[['team', 'games', 'original_cover_pct', 'handicap_cover_pct',
                        'original_rating', 'handicap_rating', 'handicap_delta']].copy()
export_df = export_df.sort_values('handicap_rating', ascending=False)

export_file = Path().resolve().parent / 'data' / 'results' / f'ncaam_handicap_{handicap}pt_ratings.csv'
export_df.to_csv(export_file, index=False)
print(f"Exported to: {export_file}")

export_df.head(20)